In [1]:
import pandas as pd
import numpy as np

driver_history = pd.read_parquet('../output_d20k_50repl/driver_generated_points')
driver_history

,start_time,end_time,start_zone,end_zone,is_moving,has_passenger,driver_id,startx,starty,endx,endy
0,0.000000,41.927466,3,3,False,False,0,881.0,96.0,881.0,96.0
1,41.927466,80.505818,3,132,True,True,0,881.0,96.0,1049.0,510.0
2,80.505818,109.557108,132,132,False,False,0,1049.0,510.0,1049.0,510.0
3,109.557108,142.751426,132,42,True,True,0,1049.0,510.0,678.0,173.0
4,142.751426,159.559577,42,42,False,False,0,678.0,173.0,678.0,173.0
...,...,...,...,...,...,...,...,...,...,...,...
1013433,1401.061903,1407.139627,75,74,True,False,19999,674.0,245.0,681.0,213.0
1013434,1407.139627,1407.139627,74,74,False,False,19999,681.0,213.0,681.0,213.0
1013435,1407.139627,1416.063465,74,75,True,True,19999,681.0,213.0,666.0,247.0
1013436,1416.063465,1416.314096,75,75,False,False,19999,666.0,247.0,666.0,247.0


In [58]:
from DriverAnimation import DriverAnimation

da = DriverAnimation(driver_history, 30, 60)

In [59]:
u = da.update()

In [60]:
import time

tic = time.time()
pos = da.update()
for p in pos:
    a = p
toc = time.time()
print(toc - tic)

0.013838052749633789
